In [14]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer

In [15]:
pd.options.display.float_format = '{:,.2f}'.format
pd.options.display.max_rows = 200
pd.options.display.max_columns = 100

In [16]:
fileData = pd.read_csv('crypto_sample.csv',names=['Code','Date','Open','High','Low','Close','Volume','VWAP','TWAP'])
fileData = fileData[fileData.Code.isin(pd.unique(fileData['Code'])[0:5])]
fileData
pivotedData = fileData.pivot(index='Date',columns='Code',values='VWAP')

In [17]:
cleanedData = pivotedData # extract the first 4 columns 
imputer = SimpleImputer(missing_values = np.nan, strategy='mean')
imputer = imputer.fit(cleanedData[:][1:])
cleanedData[:][0:] = imputer.transform(cleanedData[:][0:]) # transform and store back
cleanedData

Code,GWA_BTC,GWA_ETH,GWA_LTC,GWA_XLM,GWA_XRP
Date,,,,,
1/1/15,315.15,123.29,2.71,0.01,0.02
1/1/16,434.93,0.95,3.51,0.00,0.01
1/1/17,"1,000.38",8.42,4.48,0.00,0.01
1/1/18,"14,103.18",765.22,227.89,0.45,2.20
1/10/15,276.43,123.29,1.70,0.01,0.02
...,...,...,...,...,...
9/8/17,"4,355.51",307.01,71.25,0.02,0.22
9/9/14,471.02,123.29,5.04,0.02,0.00
9/9/15,241.57,1.23,3.00,0.00,0.01


In [18]:
delta_7 = pivotedData / pivotedData.shift(7) - 1.0
delta_14 = pivotedData / pivotedData.shift(14) - 1.0
delta_21 = pivotedData / pivotedData.shift(21) - 1.0
delta_28 = pivotedData / pivotedData.shift(28) - 1.0
melted_7 = delta_7.reset_index().melt(id_vars=['Date'], value_name='delta_7')
melted_14 = delta_14.reset_index().melt(id_vars=['Date'], value_name='delta_14')
melted_21 = delta_21.reset_index().melt(id_vars=['Date'], value_name='delta_21')
melted_28 = delta_28.reset_index().melt(id_vars=['Date'], value_name='delta_28')
delta_7
melted_7

,Date,Code,delta_7
0,1/1/15,GWA_BTC,nan
1,1/1/16,GWA_BTC,nan
2,1/1/17,GWA_BTC,nan
3,1/1/18,GWA_BTC,nan
4,1/10/15,GWA_BTC,nan
...,...,...,...
6965,9/8/17,GWA_XRP,44.10
6966,9/9/14,GWA_XRP,-0.41
6967,9/9/15,GWA_XRP,0.31
6968,9/9/16,GWA_XRP,-0.97


In [19]:
melted_dfs = [melted_7,melted_14,melted_21,melted_28]

In [36]:
finalData = pd.merge(melted_dfs[0],melted_dfs[1],on=['Date','Code'])
finalData = pd.merge(finalData,melted_dfs[2],on=['Date','Code'])
finalData = pd.merge(finalData,melted_dfs[3],on=['Date','Code'])
finalData

,Date,Code,delta_7,delta_14,delta_21,delta_28
0,1/1/15,GWA_BTC,nan,nan,nan,nan
1,1/1/16,GWA_BTC,nan,nan,nan,nan
2,1/1/17,GWA_BTC,nan,nan,nan,nan
3,1/1/18,GWA_BTC,nan,nan,nan,nan
4,1/10/15,GWA_BTC,nan,nan,nan,nan
...,...,...,...,...,...,...
6965,9/8/17,GWA_XRP,44.10,26.97,23.04,0.12
6966,9/9/14,GWA_XRP,-0.41,-0.22,-0.98,-0.03
6967,9/9/15,GWA_XRP,0.31,-0.96,0.62,0.03
6968,9/9/16,GWA_XRP,-0.97,0.22,-0.21,-0.02
